In [1]:
# Generated by Selenium IDE
##This code Scrapes Actual product description, Barcodes and Manufacturer Names using product description from dataset and generates a similarity score for the scraped product

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

  
class TestGloria():
    def setup_method(self, method):
        self.driver = webdriver.Chrome()
        self.vars = {}
    def teardown_method(self, method):
        self.driver.quit()
  
    def search(self,name):
        self.driver.find_element(By.NAME, "s").clear()
        self.driver.find_element(By.NAME, "s").send_keys(name)
        self.driver.find_element(By.NAME, "s").send_keys(Keys.ENTER)
            
    def getElements(self):
        elements = self.driver.find_elements(By.CSS_SELECTOR, ".h3.h3.product-title")
        return elements

    def getCompanyName(self):
        try:
            element = self.driver.find_element(By.CSS_SELECTOR, ".col-lg-12 > span")
            return element.text
        except:
            return ""

    
    def getBarcode(self):
        try:
            element = self.driver.find_element(By.CSS_SELECTOR, ".product-reference > span")
            return element.text
        except:
            return ""
    
    def getItemName(self):
        element = self.driver.find_element(By.CSS_SELECTOR, ".h1")
        return element.text
       
    def initialSetup(self):
        self.driver.get("https://farmaciagloria.mx/")
        self.driver.set_window_size(1440, 761)   
     

In [2]:
import unicodedata

## used to normalize data 
## like Â ,Ć ,Ê , to A , C , E

def normalize(string):
    return unicodedata.normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')

In [3]:
import re
def Tab_type_details(string):
    
    tablet_type_features = []
    ## most found tablets types are stored in below ditionary
    tab_types = {
                   "CREAM":["CREAM","CREMA","POMADA","OINTMENT"],
                   "DROPS":["GTS","GOTAS"],
                   "BOTTLE":["FRESCO","FRASCO","FRASCO AMPOLLA","SOL"],
                   "TABLET":["TABLETAS","TABLETS","TABLET","TABS","TAB","GRAGEAS"],
                   "POWDER" :["SOBRES","SOBRE"],
                   "INJECTION":["INY","I[.]V.","I[.]V","I V","IV","AMP","AMPOLLETAS","AMPOLLA","AMPULA",\
                                "VIAL","VIALS","JER","FA","F[.]A","F A","F[.]A[.]"],
                   "SUPO":["SUPOSITORIS","SUPPOSITORIES","SUPOSITORIO","SUPPOSITORIES"],
                   "CAPSULE":["COMPRIMIDOS","COMPR","COPR","COMP","CPR","CAPSULES","CAPSULAS","CAPS","CAP"]
                  }
    
    for tab_type in tab_types.keys():        
        for name in tab_types[tab_type]:
            found = re.findall(f"C?O?N?/? ?\d+(?:\.\d+)? ?{name}.? ?", string)
#             if not found:
#                 found = re.findall(f"{name} ?C?O?N?/? ?\d+(?:\.\d+)?", string)
            for i in found:
                count = re.findall("(\d+(?:\.\d+)?)",i)[0]
                tablet_type_features.append({"raw_string":i,"count":count,"found_name":name,"type":tab_type})
                string = string.replace(i," ")
    return tablet_type_features,string
    

                
            

In [4]:
## some more fine tuning:

def extractmorefeatures(string):
    extra_features = {"types":[],"counts":[],"common":[]}
    tab_types = {
                   "CREAM":["CREAM","CREMA","POMADA","OINTMENT"],
                   "DROPS":["GTS","GOTAS"],
                   "BOTTLE":["FRESCO","FRASCO","FRASCO AMPOLLA","SOL"],
                   "TABLET":["TABLETAS","TABLETS","TABLET","TABS","TAB","GRAGEAS"],
                   "POWDER" :["SOBRES","SOBRE"],
                   "INJECTION":["INY","I[.]V.","I[.]V","I V","IV","AMP","AMPOLLETAS","AMPOLLA","AMPULA",\
                                "VIAL","VIALS","JER","FA","F[.]A","F A","F[.]A[.]"],
                   "SUPO":["SUPOSITORIS","SUPPOSITORIES","SUPOSITORIO","SUPPOSITORIES"],
                   "CAPSULE":["COMPRIMIDOS","COMPR","COPR","COMP","CPR","CAPSULES","CAPSULAS","CAPS","CAP"]
                  }
    COMMON_TERMS = ["CAJA","CON","PISA"]

    ## to check for types
    for tab_type in tab_types.keys():        
        for name in tab_types[tab_type]:
            found = re.findall(f" {name} ", string)
            if not found:
                found = re.findall(f" {name}\Z",string)
            for i in found:
                extra_features["types"].append({"raw_string":i,"found_name":name,"type":tab_type})
                string = string.replace(i," ")
   
            
    # to check for quantities
    
    found = re.findall("C?O?N?/? ?\d+(?:\.\d+)?",string)
    for i in found:
        string = string.replace(i," ")
        count = re.findall("(\d+(?:\.\d+)?)",i)[0]
        extra_features["counts"].append({"raw_string":i,"count":count})
    

    for name in COMMON_TERMS:
        found = re.findall(f" {name} ", string)

        if not found:
            found = re.findall(f" {name}\Z",string)
        for i in found:
            extra_features["common"].append(i)
            string = string.replace(i," ")
    return extra_features,string
# print(cleanstring("NEO-MELUBRINA INFANTIL  /  FRASCO CON "))
    
    
    

In [5]:
import re
def findCWV(string):

    COMPOSITIONS = ["MG/ML","UI/ML","IU/ML","G/ML","U[.]I/ML","U[.]I/ML","U/ML","%","U[.]I","UI"]
    VOLUMES = ["ML","L"]
    WEIGHTS = ['GRAMS', 'GRM', 'GRS', 'GR', 'G', 'MCG', 'MC', 'MG',]
    CWV =[[],[],[]]
    for num,QUALITIES in enumerate([COMPOSITIONS,WEIGHTS,VOLUMES]):   
        for i in QUALITIES:
            ##find multiple and replace 
            regex = re.compile(f'\d+(?:\.\d+)? ?/ ?\d+(?:\.\d+)? ?{i}| ?\d+(?:\.\d+)? ?/\d+(?:\.\d+)? ?/\d+(?:\.\d+)? ?{i}')
            matches = re.findall(regex, string)

            for match in matches:
                final_string = ""

                temp = match.split("/")
                for t in temp:
                    final_string += t+f"{i} " if i not in t else t

                string = re.sub(match,final_string,string)

            
#             print(string)
            #
            temp = re.findall(f"\d+(?:\.\d+)? ?{i}[\),., ,\/]", string)
            for j in temp:
                CWV[num].append(j)
                string = string.replace(j," ")

    return CWV , string

    
findCWV("EXFORGE HCT 320/25G ")

([[], ['320G ', '25G '], []], 'EXFORGE HCT   ')

In [6]:
import re
def findName(string,CWV,tab_types,extrafeatures):

    raw_strings = []
    CWV.append(["("])

    for i in tab_types:
        CWV[-1].append(i["raw_string"])
    for feature in [extrafeatures["types"],extrafeatures["counts"]]:
        for i in feature:
            CWV[-1].append(i["raw_string"])
    min_pos_found = len(string)
    for qualities in CWV:
        for quality in qualities:
            if (quality != ''):

                position = string.find(quality)
                if position != -1:
                    min_pos_found = min(min_pos_found,position)
#     print(string[:min_pos_found],"----",string[min_pos_found:])
    return cleanstring(string[:min_pos_found])

            

In [7]:
import re
def cleanstring(text):
    text = re.sub("[\),.,\-,*,/,\(]"," ",text)
    text = text.strip()
    return text
# print(cleanstring("ASOFLON 0.4 MG C/30 CAP ( TAMSULO-SINA )wejnfe "))

In [8]:
import re

# Define the input string
string = "500 GR"


def standardize(string):
    # Define the regular expression pattern
    pattern = re.compile(r'GRAMS|GRM|GRS|GR|G')

    # Replace the substrings with 'g'
    output_string = re.sub(pattern, 'G', string)
    output_string = re.sub("[\),\ ,\(,\/]","",output_string)
    output_string = output_string.strip(".")

    # Print the output string
    return output_string

standardize("10/ 20/25 MG.")        

'102025MG'

In [9]:
def Extract_Details(text):
    text = (" ".join(text.split())).strip()
    text = text +" "
    text = normalize(text).upper()
    string = normalize(text).upper()
    details = {}
    length = len(string)
    CWV,string = findCWV(string)
    for key, i in enumerate(["compositions","weights","volumes"]):
        details[i]=CWV[key]
    details["tab_type_details"],string= Tab_type_details(string)
    extrafeatures,string = extractmorefeatures(string)
    details["name"] = findName(text,CWV,details["tab_type_details"],extrafeatures)
    string = string.replace(details["name"],"")
    string = (" ".join(string.split())).strip()
    string = cleanstring(string)
    details["extrafeatures"] = extrafeatures
    for quant in ["weights","volumes","compositions"]:
        for key,i in enumerate(details[quant]):
            details[quant][key] = standardize(i)
    types = set()
    counts = set()

            
    for detail in details["tab_type_details"]:
        if len(details["tab_type_details"]) != 0:
            types.add(detail["type"])
            counts.add(detail["count"])
    for detail in details['extrafeatures']['types']:
        types.add(detail["type"])
    for detail in details['extrafeatures']['counts']:
        counts.add(detail["count"])
    details["features"] = {"types":types,"counts":counts,"extra":set(string.split())}
    details.pop('tab_type_details')
    details.pop('extrafeatures')    
    return details

Extract_Details("NIQUITIN 21 MG C/7 PARCHES CON PARA")
Extract_Details("EXFORGE HCT 10MG/320MG/25MG CON 28 COMPRIMIDOS TAB")
Extract_Details("SIMPLE 50 ML")

{'compositions': [],
 'weights': [],
 'volumes': ['50ML'],
 'name': 'SIMPLE',
 'features': {'types': set(), 'counts': set(), 'extra': set()}}

In [10]:
def similarity_type(target_type,src_type):
    similarity_dict ={
        "SUPOCREAM" : 0.24,
        "SUPODROPS" : 0.3,
        "SUPOBOTTLE" : 0.24,
        "SUPOTABLET" : 0.35,
        "SUPOPOWDER" : 0.29,
        "SUPOINJECTION" : 0.2,
        "SUPOCAPSULE" : 0.39,
        "CREAMDROPS" : 0.59684736728668213,
        "CREAMBOTTLE" : 0.5013158917427063,
        "CREAMTABLET" : 0.33642226457595825,
        "CREAMPOWDER" : 0.48533108830451965,
        "CREAMINJECTION" : 0.49305346608161926,
        "CREAMCAPSULE" : 0.3910931646823883,
        "DROPSBOTTLE" : 0.7995974540710449,
        "DROPSTABLET" : 0.603466272354126,
        "DROPSPOWDER" : 0.583532989025116,
        "DROPSINJECTION" : 0.6436945199966431,
        "DROPSCAPSULE" : 0.6268869638442993,
        "BOTTLETABLET" : 0.5191242694854736,
        "BOTTLEPOWDER" : 0.6567681431770325,
        "BOTTLEINJECTION" : 0.92448949813843,
        "BOTTLECAPSULE" : 0.5893133878707886,
        "TABLETPOWDER" : 0.5272809267044067,
        "TABLETINJECTION" : 0.5583986043930054,
        "TABLETCAPSULE" : 0.92078971862793,
        "POWDERINJECTION" : 0.6065213680267334,
        "POWDERCAPSULE" : 0.5563412308692932,
        "INJECTIONCAPSULE" : 0.6149609088897705
    }
    target_type = target_type.strip()
    src_type = src_type.strip()
#     print(f'{target_type}{src_type}')
    if(f'{target_type}{src_type}' in similarity_dict):
        return similarity_dict[f"{target_type}{src_type}"]
    if(f'{src_type}{target_type}' in similarity_dict):
        return similarity_dict[f"{src_type}{target_type}"]
        
    return 0


# similarity("TABLET","CREAM")

In [11]:
import jellyfish
def nameScore(s1, s2):
    tokens1 = set(s1.split())
    tokens2 = set(s2.split())
    try:
        jaccard_sim = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))
        levenshtein_dist = jellyfish.levenshtein_distance(s1, s2)
    #     print(jaccard_sim,levenshtein_dist)
        similarity = min(0.8*jaccard_sim + 0.2* (2 - (levenshtein_dist / max(len(s1), len(s2)))),1 )

        return similarity
    except :
        
        return 0.5


In [12]:
def surityscore(src,target):
    src = Extract_Details(src)
    target = Extract_Details(target)
    score = 0
    maxscore = 0
    weightages ={
        "name":3000,
        "weights":700,
        "volumes":700,
        "compositions":200,
        "types":1000,
        "counts":1000,
        "extra":70,
    }

    maxscore+=weightages["name"] 
    namescore = nameScore(src["name"] , target["name"])
    score +=namescore * weightages["name"]
    

    
    for quantity in ["weights","volumes","compositions"]:
        if len(src[quantity]) != 0 or len(target[quantity]) !=0:
#             print(src[quantity],target[quantity])
            for weight in src[quantity]:
                maxscore+= weightages[quantity]
                if weight in target[quantity]:  score += weightages[quantity]
                else: 

                    score -= weightages[quantity]
            for weight in target[quantity]:
                maxscore+= weightages[quantity]
                if weight in src[quantity]:  score += weightages[quantity]
                else: score -= weightages[quantity]

    for params in ["types","counts"]:
        
        if (len(src["features"][params]) == 0  and len(target["features"][params] ) !=0) or \
           (len(src["features"][params]) != 0 and len(target["features"][params] ) ==0) :
            maxscore+= weightages[params]
        elif (len(src["features"][params]) != 0 and len(target["features"][params] )) !=0:
            maxscore+= weightages[params] * (len(src["features"][params]) + len(target["features"][params]))
#             print(maxscore,score)
            for src_type in src["features"][params]:
                if src_type in target["features"][params]:
                    score+= weightages[params]
                else: 
                    score+= max([similarity_type(src_type,i) for i in target["features"][params]]) * weightages[params]
            for target_type in target["features"][params]:
                if target_type in src["features"][params]:
                    score+= weightages[params]
                else: 
                    score+= max([similarity_type(target_type,i) for i in src["features"][params]]) * weightages[params]

                    
    for i in src["features"]["extra"]:
        maxscore += weightages["extra"]
        if i in target["features"]["extra"]: score +=weightages["extra"]
    for i in target["features"]["extra"]:
        maxscore += weightages["extra"]
        if i in src["features"]["extra"]: score +=weightages["extra"]       
    return score * 100/maxscore


    

In [13]:
import itertools
def generatepossiblestrings(word_list):
    all_strings = set()

    # Generate all possible combinations of the words in the list
    for i in range(1, len(word_list)+1):
        for combination in itertools.combinations(word_list, i):
            all_strings.add(" ".join(combination))
    # Print all the generated strings
    return all_strings
# print(generatepossiblestrings("VANCOCIN CP".split()))

In [14]:
"""
    to go with:- almetric
    AQUADRAN GEL 10G MCA SOPHIA
    AQUASOL E (VITAMINA E) SOL 30ML
    AQYNZIO 0.5/300MG C/1 CAP
    ATEMPERATOR 500 MG. C/20 GRAG
    AVAPRO 150 MG. C/14 TAB,AVAPRO 150 MG Caja C/ 14 - WHY ONLY 71
    BENERVA 300 MG. C/20 GRAG
    BIKTARVY 50MG/200MG/25MG C/30 TAB , BIKTARVY 50/200/25MG C/30 TAB-- TO CONSIDER
    CONSIDER EVEN COMPANY NAME 
    CLOPIDOGREL 75MG C/28 TAB RETAQ,RETAQ 75 MG Caja C/ 28 Tabletas -- how correct
    BUSCAPINA COMP. C/36 GRAGEAS - all buscapina 
    CO PLAVIX 75/100MG C/28. COMP for coplavix
    ASOFLON 0.4 MG C/30 CAP ( TAMSULOSINA ).
    AVELOX 7D 400MG. C/7 CAP. -- AVELOX 400 MG CON 7 TABLETAS
    CANESTEN V CREMA 20 GR
    ALDARA CREMA  C/12 SOBRES

    NEO-MELUBRINA INFANTIL 250 MG/5 ML FRASCO CON 100 ML
    
    
    
    
    CALCITRIOL 0.25MCG C/30 CAP,ROCALTROL 0.25 MCG Caja C/ 30 Capsulas,7613326005852,Calcitriol,75.0
"""




src = "ATOZET 10MG/20 MG C/30 TABLETAS"
target = "vana 0.5 MG Caja C/ 2 Tabletas"
print(Extract_Details(src),"\n",Extract_Details(target))
print(surityscore(src,target))

{'compositions': [], 'weights': ['10MG', '20MG'], 'volumes': [], 'name': 'ATOZET', 'features': {'types': {'TABLET'}, 'counts': {'30'}, 'extra': set()}} 
 {'compositions': [], 'weights': ['0.5MG'], 'volumes': [], 'name': 'VANA', 'features': {'types': {'TABLET'}, 'counts': {'2'}, 'extra': set()}}
5.4945054945054945


In [15]:
from playsound import playsound
import pandas as pd
inputfile = "input.csv"
outfile = open("output.csv",'a')
startline = 5
refresh = 1
required_similarity = 63
# Read CSV file into a Pandas DataFrame
df = pd.read_csv(inputfile,header=None)
total_len = len(df.index)

scraper = TestGloria()
scraper.setup_method(None)
scraper.initialSetup()
scraper.driver.implicitly_wait(3)

# try:
count = 0
found = 0
for element in df.iloc[startline - 1:,0]:
#     for element in newset:
    count +=1
    scraper.search(element)
#         time.sleep(2)
    fullnames = scraper.getElements()
    similar = None
    product_name = None
    ## do similarity things and store similar into similar
    print(f"Line {startline+count-1}: ",end = "")
    if fullnames:
#             print([i.text for i in fullnames])
        surity_indexes = [surityscore(element,target.text) for target in fullnames]
        max_surity_indexes = max(surity_indexes) if len(fullnames)>0 else 0
#             print(max_surity_indexes)
        if max_surity_indexes >= required_similarity:
            similar = surity_indexes.index(max_surity_indexes)
            product_name = fullnames[similar].text

    if product_name!= None:
        fullnames[similar].click()
        time.sleep(1)
        upc,company = scraper.getBarcode(),scraper.getCompanyName()
        found +=1
        product_name = " ".join(product_name.split(","))
        company = " ".join(company.split(","))
        print(f"{normalize(element)},{normalize(product_name)},{upc},{normalize(company)},{max_surity_indexes}")
        outfile.write(f"{normalize(element)},{normalize(product_name)},{upc},{normalize(company)},{max_surity_indexes}\n")
    else:
        print(element,"Notfound")
        outfile.write(f"{normalize(element)}\n")
    if count % refresh == 0:
        print(f"\n ******** {found} found, out of {count} , total {total_len} ********\n")
#             scraper.driver.refresh()
outfile.close()
scraper.teardown_method(None)
# except Exception as e:
#     print(e)
#     outfile.close()
#     scraper.teardown_method(None)




Line 5: 3-A OFTENO SOL 0.1 % GTS 5 ML Notfound

 ******** 0 found, out of 1 , total 674 ********

Line 6: ANTICOAGULANTE ACD CELLEX 750 ML Notfound

 ******** 0 found, out of 2 , total 674 ********

Line 7: 5-FLUOROURACILO 1G/20ML Notfound

 ******** 0 found, out of 3 , total 674 ********

Line 8: SOL.ELECTROL HEMOFILT.COMP X5L Notfound

 ******** 0 found, out of 4 , total 674 ********

Line 9: A-M-K 100 MG/2 ML AMPOLLETA CON 2 ML  Notfound

 ******** 0 found, out of 5 , total 674 ********

Line 10: A-M-K 500 MG/2 ML AMPOLLETA DE 2 ML  Notfound

 ******** 0 found, out of 6 , total 674 ********

Line 11: A-S COR GOTAS 24 ML Notfound

 ******** 0 found, out of 7 , total 674 ********

Line 12: A.I.N.E GEL X 30 GRS Notfound

 ******** 0 found, out of 8 , total 674 ********

Line 13: ABASAGLAR KWIKPEN 100UI/ML Notfound

 ******** 0 found, out of 9 , total 674 ********

Line 14: ABATACEPT 250MG Notfound

 ******** 0 found, out of 10 , total 674 ********

Line 15: ABCIXIMAB 10MG/5ML Notfound


Line 82: ACOEXCEL (PACLITAXEL) 30MG/5ML SOLUCION INYECTABLE Notfound

 ******** 10 found, out of 78 , total 674 ********

Line 83: ACTEMRA 80 MG / 4 ML Notfound

 ******** 10 found, out of 79 , total 674 ********

Line 84: ACTILYSE 50 MG 2 FRASCOS AMPULA CON POLVO Y 2 AMPOLLETAS CON 50 ML DE DI Notfound

 ******** 10 found, out of 80 , total 674 ********

Line 85: ACTILYSE 50 MG F.A 50 ML C/2 PZAS Notfound

 ******** 10 found, out of 81 , total 674 ********

Line 86: ACTINOMICINA D 0.5 MG Notfound

 ******** 10 found, out of 82 , total 674 ********

Line 87: ACUAFIL OFT GTS 15 ML Notfound

 ******** 10 found, out of 83 , total 674 ********

Line 88: ACULAREN 0.5% SOL/OFT.5 ML Notfound

 ******** 10 found, out of 84 , total 674 ********

Line 89: ADALAT OROS 30 MG 10 TABLETAS DE LIBERACION PROLONGADA Notfound

 ******** 10 found, out of 85 , total 674 ********

Line 90: ADAREX (PREDNISOLONA) 1MG/ML SOLUCION Notfound

 ******** 10 found, out of 86 , total 674 ********

Line 91: ADAREX 3M

Line 148: AKATONE 30ML 3% FA Notfound

 ******** 27 found, out of 144 , total 674 ********

Line 149: ALACRAMYN 150 U FRASCO AMPULA CON LIOFILIZADO Notfound

 ******** 27 found, out of 145 , total 674 ********

Line 150: ALACRAMYN AMP. 5 ML ( ALACRAN ) Notfound

 ******** 27 found, out of 146 , total 674 ********

Line 151: ALBATRINA 100MG/100ML FCO 60ML (LORATADINA).(G.I) Notfound

 ******** 27 found, out of 147 , total 674 ********

Line 152: ALBENDAZOL SUSP ORAL FRASCO 20 ML,ALOSOL FRASCO SOLUCION SPRAY CON 20 ML,7501165001800,SANFER,63.91437308868502

 ******** 28 found, out of 148 , total 674 ********

Line 153: ALBOTHYL SOL. 12 ML Notfound

 ******** 28 found, out of 149 , total 674 ********

Line 154: ALBUMINA 20% 10 ML Notfound

 ******** 28 found, out of 150 , total 674 ********

Line 155: ALBUMINA 5% 500 CC Notfound

 ******** 28 found, out of 151 , total 674 ********

Line 156: ALBUMINA HUM. GRIFOLS 20% 50 ML,ALBUMINA HUMANA 20% BEHRING DE 50 ML,7501088589553,CSL BEHRING,64.

Line 223: AMIFARIN 250MG/5ML 60ML (DICLOXACILINA),FOSFOCIL 250MG/5ML SUSPENSION CON 60 ML,7501314701742,SENOSIAIN,67.43869209809264

 ******** 38 found, out of 219 , total 674 ********

Line 224: AMIKACINA 500 MG AMP ( KENER ),AMIKACINA 500MG/2ML CON 2 AMPOLLETAS,7501349021488,AMSA,69.1747572815534

 ******** 39 found, out of 220 , total 674 ********

Line 225: AMIKACINA IM 500MG AMP1X2 MCA AMSA Notfound

 ******** 39 found, out of 221 , total 674 ********

Line 226: AMINOACIDOS 10% X 500 CC Notfound

 ******** 39 found, out of 222 , total 674 ********

Line 227: AMINOACIDOS 5% X 500 ML Notfound

 ******** 39 found, out of 223 , total 674 ********

Line 228: AMINOACIDOS 8% X 500 ML Notfound

 ******** 39 found, out of 224 , total 674 ********

Line 229: AMINOACIDOS AL 15 % 500 ML Notfound

 ******** 39 found, out of 225 , total 674 ********

Line 230: AMINOACIDOS PEDIATRICOS 10 % X 250 ML Notfound

 ******** 39 found, out of 226 , total 674 ********

Line 231: AMINOACIDOS RAMIFICADOS 1

Line 293: ANTIRRABICA (VERORAB) Notfound

 ******** 51 found, out of 289 , total 674 ********

Line 294: ANTITOXINA TETANICA PARA USO HUMANO Notfound

 ******** 51 found, out of 290 , total 674 ********

Line 295: ANTITROMBINA 500 UI Notfound

 ******** 51 found, out of 291 , total 674 ********

Line 296: ANTIVIP MYN F.A SOL Notfound

 ******** 51 found, out of 292 , total 674 ********

Line 297: ANTIVON 4MG/2ML AMP  Notfound

 ******** 51 found, out of 293 , total 674 ********

Line 298: ANTIVON 8MG/4ML  Notfound

 ******** 51 found, out of 294 , total 674 ********

Line 299: APIROVIN (ROPIVACAINA) 150MG/20ML Notfound

 ******** 51 found, out of 295 , total 674 ********

Line 300: APIROVIN (ROPIVACAINA) 7.5MG/ML AMP 20ML Notfound

 ******** 51 found, out of 296 , total 674 ********

Line 301: APIXABAN 2.5MG Notfound

 ******** 51 found, out of 297 , total 674 ********

Line 302: APIXABAN 5 MG Notfound

 ******** 51 found, out of 298 , total 674 ********

Line 303: APREPITANT 125/80 MG

Line 363: ATOSIBAN 7.5MG/MLX0.9ML  Notfound

 ******** 66 found, out of 359 , total 674 ********

Line 364: ATOSIBAN 7.5MG/MLX5ML INFUS  Notfound

 ******** 66 found, out of 360 , total 674 ********

Line 365: ATRACURIO BESILATO 25MG/2.5 ML Notfound

 ******** 66 found, out of 361 , total 674 ********

Line 366: ATRO OFTENO 1% 15 ML Notfound

 ******** 66 found, out of 362 , total 674 ********

Line 367: ATROPINA 1 MG/1 ML AMP Notfound

 ******** 66 found, out of 363 , total 674 ********

Line 368: ATROPINA JERINGA PRELL 1MG/10ML (0.1MG/ML)  Notfound

 ******** 66 found, out of 364 , total 674 ********

Line 369: ATROPINA SULF 0.1 AMP 1 ML  Notfound

 ******** 66 found, out of 365 , total 674 ********

Line 370: ATROPINA SULFATO 1% Notfound

 ******** 66 found, out of 366 , total 674 ********

Line 371: ATROPINE SULFATE 1MG-ML SDV 25X1M Notfound

 ******** 66 found, out of 367 , total 674 ********

Line 372: ATROVENT 250 MCG/1 ML FRASCO CON 20 ML,ATROVENT 250 MCG SOLUCION FRASCO CON 20

Line 433: BAPENTOR 500 MG FCO AMP ( MEROPENEM ) Notfound

 ******** 81 found, out of 429 , total 674 ********

Line 434: BARIO SULFATO SUSPENSION 95% X 340 G Notfound

 ******** 81 found, out of 430 , total 674 ********

Line 435: BARMICIL COMP CREMA 30 G Notfound

 ******** 81 found, out of 431 , total 674 ********

Line 436: BARMICIL COMPUESTO CREMA 40G MCA SONS,BARMICIL COMPUESTO CREMA EN TUBO DE 40 G,7502001166066,,94.81481481481481

 ******** 82 found, out of 432 , total 674 ********

Line 437: BASILIXIMAB 20 MG/5 ML VIAL Notfound

 ******** 82 found, out of 433 , total 674 ********

Line 438: BAYCUTEN CREMA 30 G,BAYCUTEN 30 G CREMA,7501318649071,BAYER,100.0

 ******** 83 found, out of 434 , total 674 ********

Line 439: BAYCUTEN N CREMA 35 GR,BAYCUTEN N 35 G CREMA,7501318646360,BAYER,100.0

 ******** 84 found, out of 435 , total 674 ********

Line 440: BAAO COLOIDE PVO 90 GR SOBRE Notfound

 ******** 84 found, out of 436 , total 674 ********

Line 441: BCG 30 MG (MYCOBACTERIUM BO

Line 505: BIOGAIA 0.45G C/10 TAB MASTICABLES Notfound

 ******** 92 found, out of 501 , total 674 ********

Line 506: BIOGAIA PROD C/30 TAB S/AZUCAR SABOR MENTA,MELOX PLUS CON 30 TABLETAS MASTICABLES SABOR MENTA,7501165002920,SANOFI AVENTIS,66.48865153538051

 ******** 93 found, out of 502 , total 674 ********

Line 507: BIOGENFINE C/10 MCA BIOGENTEC Notfound

 ******** 93 found, out of 503 , total 674 ********

Line 508: BIOMESINA 10 MG GRAG (BUTILHIOSCINA) Notfound

 ******** 93 found, out of 504 , total 674 ********

Line 509: BIOMESINA COMPUESTA 10/250MG GRAGEA Notfound

 ******** 93 found, out of 505 , total 674 ********

Line 510: BIOMESINA COMPUESTA GRAGEAS 10MG /250MG C/10 ( NAXCA ) Notfound

 ******** 93 found, out of 506 , total 674 ********

Line 511: BIOMESINA GRAGEAS 10MG C/10 ( NAXCA ) Notfound

 ******** 93 found, out of 507 , total 674 ********

Line 512: BIOMICS SUSP 50 ML Notfound

 ******** 93 found, out of 508 , total 674 ********

Line 513: BIOMIVIT - 400 9MESES C/

Line 575: BUDESONIDA + FORMOTEROL 80/4.5 MCG Notfound

 ******** 106 found, out of 571 , total 674 ********

Line 576: BUDESONIDA 200MCG X DOSIS HFA AEROSOL Notfound

 ******** 106 found, out of 572 , total 674 ********

Line 577: BUPIVACAINA 0.75% SIMPLE 30ML HOSPIRA Notfound

 ******** 106 found, out of 573 , total 674 ********

Line 578: BUPIVACAINA 05MG X 10ML Notfound

 ******** 106 found, out of 574 , total 674 ********

Line 579: BUPIVACAINA 5 MG/ML X 10ML Notfound

 ******** 106 found, out of 575 , total 674 ********

Line 580: BUPIVACAINA HIPERBARICA 7.5MG/ML X 2ML Notfound

 ******** 106 found, out of 576 , total 674 ********

Line 581: BUPIVACAINA SIMPLE 50% 30 ML ANDROMEDICS Notfound

 ******** 106 found, out of 577 , total 674 ********

Line 582: BUPIVACAINE HCL PF 0.75% SDV 25X30 ML Notfound

 ******** 106 found, out of 578 , total 674 ********

Line 583: BUPRENORFINA 10MCG/H CONTROLADO 2 UN Notfound

 ******** 106 found, out of 579 , total 674 ********

Line 584: BUPRENO

Line 637: ATENOLOL 50 MG C/28 TAB ( KENER ),TENORMIN 50 MG CAJA CON 28 TABLETAS,7501324402585,SANDOZ,73.49468713105077

 ******** 135 found, out of 633 , total 674 ********

Line 638: ATOMOXETINA 40MG C/14 TAB,MOXAZYD 40 MG CAJA CON 14 CAPSULAS,8901120011742,ZYDUS,71.490664296336

 ******** 136 found, out of 634 , total 674 ********

Line 639: ATORVASTATINA 10 TABS 40MG,ATORVASTATINA 40 MG CON 10 TABLETAS,7501349028913,AMSA,84.66857817507169

 ******** 137 found, out of 635 , total 674 ********

Line 640: ATORVASTATINA 10MG C/20 TAB Notfound

 ******** 137 found, out of 636 , total 674 ********

Line 641: ATORVASTATINA 20 MG C/30 TAB,ATORVASTATINA 20 MG CON 10 TABLETAS,7501349024526,AMSA,76.19047619047619

 ******** 138 found, out of 637 , total 674 ********

Line 642: AURICAM (MELOXICAM 15MG) CAJA /10 TAB,MAVICAM 15 MG CON 10 TABLETAS,0785120754087,,74.88615280823073

 ******** 139 found, out of 638 , total 674 ********

Line 643: AVALRAAM VALSARTAN 80MG C/30 COMP,DIOVAN 80 MG CON 30 